In [ ]:
from Bio.PDB import *
parser = PDBParser()
from tqdm import tqdm

In [ ]:
import torch
from torch_geometric.data import *

In [ ]:
def get_one_hot_dictionary(keys):
    vector_size = len(keys)
    dictionary = {}
    for x_index, x in enumerate(keys):
        one_hot = [0] * (vector_size-1)
        one_hot.insert(x_index,1)
        dictionary[x] = one_hot
    return dictionary

In [ ]:
keys = Polypeptide.d3_to_index.keys()
hot_dick = get_one_hot_dictionary(keys)

In [ ]:
def get_residue_list(chain):
    return [res for res in chain if res.resname in Polypeptide.d3_to_index.keys()]

def get_covalent_edges(residues):
    return [[index, index-1] for index, x in enumerate(residues) if x._id[1] == ((residues[index-1]._id[1])+1)]

def check_proximity(residue_one, residue_two, max_distance = 8) :
    """Returns the C-alpha distance between two residues"""
    '''C BETA IS PREFEREABLE BUT NEEDS GLYCINE ENCODING'''
    diff_vector  = residue_one["CA"].coord - residue_two["CA"].coord
    euclidian_distance = numpy.sqrt(numpy.sum(diff_vector * diff_vector))
    if euclidian_distance < max_distance:     
        return True
    else:
        return False

In [ ]:
def get_graph(residues):
    processed_residues = get_residue_list(residues)
    covalent_edges = get_covalent_edges(processed_residues)
    proximal_edges = [] 
    for x_index, x in enumerate(processed_residues):
        for y_index, y in enumerate(processed_residues):
            if y_index >= x_index + 5: 
                if y > x:
                    if check_proximity(x, y):
                        proximal_edges.append([x_index,y_index])

    all_edges = covalent_edges + proximal_edges
    node_features = [hot_dick[res.resname] for res in processed_residues]
    edge_index = torch.tensor(all_edges, dtype=torch.long)
    node_data = torch.tensor(node_features, dtype=torch.float)
    # print(node_data.shape)
    graph = Data(x = node_data, edge_index=edge_index.t().contiguous())
    return graph

In [ ]:
graph_list = []

for oof in tqdm(range(2000)):
    structure = parser.get_structure('ATP', '1ATP.pdb')
    model = structure[0]
    residues = structure.get_residues()
    graph = get_graph(residues)
    graph_list.append(graph)

loader = DataLoader(graph_list, batch_size=200)